# <span style="color:blue"> Scientific Data Analysis Final Project </span>

**Project Group #:** &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;10<br>
**Group Members:** &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Saffron Birch(100795118), Victor Ma(100789474), Nicholas Panait(100776699), and Ji Long Alden Chan(100783638)<br><br>

## <span style="color:blue"> Introduction: </span>
**Why did you choose this dataset over others?**<br>
>The main reason why we choose this dataset over others was because the dataset is on a topic that we found interesting. This combined with the fact that the dataset contains 20058 points of data with 16 attributes, makes it a very useful dataset for analysis. Another reason we choose to do a dataset on chess rather than something else is the value of data analysis in something as complex as chess. In chess, there is more variation the more moves you make, because of this it is easy to see which moves are better earlier on.

**What are the data analysis questions you want to use this dataset for?**<br>
>1. Are the chances of winning the game greater if you are playing white? If so, why is that?

>2. Does the number of moves affect the outcome of the game? If the number of moves played goes over the average number of moves, what is the most likely outcome of the match? What if the number of moves is below average?

>3. How do the following victory statuses correlate with rating? Find the correlation coefficient and determine whether the relationship is strong/weak and positive/negative. What conclusions can be drawn from the data collected?<br>
>  a) Checkmate<br>
>  b) Resignation<br>
>  c) Timeout <br>

>4. List the top 5 highest win rate openings, what are their win rates? Do their win rates vary depending on piece colour? List the top 5 players that use the combination of these openings the most, what are their win rates and rating? What conclusions can be drawn from the data collected? 

>5. What is the relationship between the least and most common chess openings and their play rates? Does the trend indicate increasing or decreasing volatility in victory results? What conclusions can be drawn from the data collected?

>6. Is there a correlation between how many variations an opening has and how often it is played? 

**What did each team member do?**

## <span style="color:blue"> Description of the Data: </span>
**Dataset Link:** &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; https://www.kaggle.com/datasnaek/chess/<br>
**Dataset Name:** &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Chess Game Dataset(Lichess)<br>
**Dataset Description:** This dataset holds the records of over 20000 games of chess played on [Lichess](https://lichess.org/) collected using the [Lichess API](https://github.com/ornicar/lila).<br>
**Dataset Attributes:** <br>
>**id** : The ID associated with the game in Lichess's database of games.<br>
>**rated** : Whether the game played was casual or worth rating.<br>
>**created_at** : The time in seconds that the game was created.<br>
>**last_move_at** : The time in seconds that the last move of the game was played and it finished.<br>
>**turns** : The total number of turns played in the game.<br>
>**victory_status** : How the game was won, by timeout, checkmate, resignation or if it was a draw.<br>
>**winner** : The colour pieces won the match or whether it was a draw.<br>
>**increment_code** : The time alloted for the match in the form: *Time limit per side in minutes* : *Seconds added to the time limit per move*.<br>
>**white_id** : The username of the person playing white.<br>
>**white_rating** : The rating of the person playing white.<br>
>**black_id** : The username of the person playing black.<br>
>**black_rating** : The rating of the person playing black.<br>
>**moves** : The moves played during that game in standard chess notation.<br>
>**opening_eco** : The unique standardized code given to every opening.<br>
>**opening_name** : The name of the opening played.<br>
>**opening_ply** : The number of moves in this specific opening.<br>

In [7]:
import csv
import re
from functools import reduce
import numpy as np

# this aux function reads the CSV file and returns the data in a Python dictionary
def get_data_csv():
    collection = []
    with open('games.csv', 'r') as f:
        for line in csv.DictReader(f):
            collection.append(line)
        return collection
        
# the data    
data = get_data_csv()

## <span style="color:blue"> Cleaning and Preprocessing of Data:</span>

**Unrated games are not qualitative games as you could have people testing things out or having fun and thus may skew results or act as outliers so we will remove them.**
> How many of these games were rated?<br>
> How many of these games were unrated?

In [8]:
rated_games = list(filter(lambda x : x["rated"] == "TRUE" or x["rated"] == "True", data))
print(str(len(rated_games)) + " games were rated.")
unrated_games = list(filter(lambda x : x["rated"] == "FALSE" or x["rated"] == "False", data))
print(str(len(unrated_games)) + " games were unrated.")
# Remove unrated as the quality of the games is low
data = rated_games
print("Number of games left after removing unrated games: "+str(len(data)))

16155 games were rated.
3903 games were unrated.
Number of games left after removing unrated games: 16155


**Games with large rating differences are also not qualitative games as someone more skilled could play pretty much any opening and still win simply because the skill gap is too large**.

In [9]:
# Remove games with a rating gap of more than 500
data = list(filter(lambda x: int(x["white_rating"])-int(x["black_rating"]) >= -500 and int(x["white_rating"])-int(x["black_rating"]) <= 500, data))
print("Number of games left after processing rating gap: "+str(len(data)))

Number of games left after processing rating gap: 15451


**There are odd cases in the dataset where the capitalizaiton of the strings are not consistent so we will lowercase all strings to make them easier to analyze.**

In [10]:
# Lowercase all of victory status and winner for easier processing
map(lambda x: x["winner"], data)
map(lambda x: x["victory_status"].lower(), data)

## <span style="color:blue"> Analyzing Our Data:</span>
>1.a How many chess games have been recorded in this dataset?<br>
>1.b What do the first 3 records look like?

In [11]:
print("There are " + str(len(data)) + " games recorded in this dataset.")
print("\nRecord 1:")
print(data[0])
print("\nRecord 2:")
print(data[1])
print("\nRecord 3:")
print(data[2])

There are 15451 games recorded in this dataset.

Record 1:
{'id': 'l1NXvwaE', 'rated': 'TRUE', 'created_at': '1.50413E+12', 'last_move_at': '1.50413E+12', 'turns': '16', 'victory_status': 'resign', 'winner': 'black', 'increment_code': '5+10', 'white_id': 'a-00', 'white_rating': '1322', 'black_id': 'skinnerua', 'black_rating': '1261', 'moves': 'd4 Nc6 e4 e5 f4 f6 dxe5 fxe5 fxe5 Nxe5 Qd4 Nc6 Qe5+ Nxe5 c4 Bb4+', 'opening_eco': 'B00', 'opening_name': 'Nimzowitsch Defense: Kennedy Variation', 'opening_ply': '4'}

Record 2:
{'id': 'mIICvQHh', 'rated': 'TRUE', 'created_at': '1.50413E+12', 'last_move_at': '1.50413E+12', 'turns': '61', 'victory_status': 'mate', 'winner': 'white', 'increment_code': '5+10', 'white_id': 'ischia', 'white_rating': '1496', 'black_id': 'a-00', 'black_rating': '1500', 'moves': 'e4 e5 d3 d6 Be3 c6 Be2 b5 Nd2 a5 a4 c5 axb5 Nc6 bxc6 Ra6 Nc4 a4 c3 a3 Nxa3 Rxa3 Rxa3 c4 dxc4 d5 cxd5 Qxd5 exd5 Be6 Ra8+ Ke7 Bc5+ Kf6 Bxf8 Kg6 Bxg7 Kxg7 dxe6 Kh6 exf7 Nf6 Rxh8 Nh5 Bxh5 Kg5 Rxh7 K

>2.a How many moves was the longest game?<br>
>2.b How many moves was the shortest game?<br>

In [12]:
# print(data[0]["turns"])
most_moves = [];
for i in data:
#     print(i["turns"])
    most_moves.append(int(i["turns"]))
most_moves.sort()
print("The longest game had " + str(most_moves[len(most_moves)-1]) + " move(s).")
print("The shortest game had " + str(most_moves[0]) + " move(s).")

The longest game had 349 move(s).
The shortest game had 1 move(s).


>3.a How many games were won by checkmate?<br>
>3.b How many games were won by resignation?<br>
>3.c How many games were won by timeout?<br>
>3.d How many games were draws?<br>

In [13]:
checkmate_games = list(filter(lambda x : x["victory_status"] == "mate", data))
print(str(len(checkmate_games)) + " games were won by checkmate.")
resignation_games = list(filter(lambda x : x["victory_status"] == "resign", data))
print(str(len(resignation_games)) + " games were won by resignation.")
timeout_games = list(filter(lambda x : x["victory_status"] == "outoftime", data))
print(str(len(timeout_games)) + " games were won by timeout.")
draw_games = list(filter(lambda x : x["victory_status"] == "draw", data))
draw_games += list(filter(lambda x : x["victory_status"] == "outoftime" and x["winner"] == "draw", data))
print(str(len(draw_games)) + " games were draws.")

4870 games were won by checkmate.
8584 games were won by resignation.
1321 games were won by timeout.
714 games were draws.


>4.a Of all the games recorded in this data set, how many did white win?<br>
>4.b Of all the games recorded in this data set, how many did black win?<br>
>4.c Of all the games recorded in this data set, how many were a draw?

In [14]:
white_won = list(filter(lambda x : x["winner"] == "white", data))
print(str(len(white_won)) + " games were won by white.")
black_won = list(filter(lambda x : x["winner"] == "black", data))
print(str(len(black_won)) + " games were won by black.")
tie = list(filter(lambda x : x["winner"] != "white" and x["winner"] != "black", data))
print(str(len(tie)) + " games were a tie.")

7698 games were won by white.
7039 games were won by black.
714 games were a tie.


>5.a How many games had a time increment?<br>
>5.b Of the games that did have an increment, which increment was the most popular?<br>
>5.c How many games didn't have a time increment?<br>
>5.d Of the games that didn't have an increment, what was the most used time limit?

In [15]:
# 5.a
def has_increment(increment):
    time = increment["increment_code"].split("+")
    if (time[1] == "0"):
        return False
    return True
increment_games = list(filter(has_increment, data))
print(str(len(increment_games)) + " games had an increment.")

# 5.b
increments = []
counters = []
for i in increment_games:
    increment = i["increment_code"].split("+")
    new = True
    for j in range(len(increments)):
        if (increments[j] == increment[1]):
            counters[j] += 1
            new = False
            break
    if new:
        increments.append(increment[1])
        counters.append(1)
biggest = counters[0];
biggest_index = 0;
for i in range(1, len(counters)):
    if counters[i] > biggest:
        biggest = counters[i]
        biggest_index = i
print("A " + str(increments[biggest_index]) + " second increment was the most popular with " + str(biggest) + " games using it.")

# 5.c
def no_increment(increment):
    time = increment["increment_code"].split("+")
    if (time[1] == "0"):
        return True
    return False
no_increment_games = list(filter(no_increment, data))
print(str(len(no_increment_games)) + " games did not have an increment.")

#5.d
times = []
counters = []
for i in no_increment_games:
    time = i["increment_code"].split("+")
    new = True
    for j in range(len(times)):
        if (times[j] == time[0]):
            counters[j] += 1
            new = False
            break
    if new:
        times.append(time[0])
        counters.append(1)
biggest = counters[0];
biggest_index = 0;
for i in range(1, len(counters)):
    if counters[i] > biggest:
        biggest = counters[i]
        biggest_index = i
print(str(times[biggest_index]) + " minute time limits were the most popular with " + str(biggest) + " games using it.")

6569 games had an increment.
A 5 second increment was the most popular with 1551 games using it.
8882 games did not have an increment.
10 minute time limits were the most popular with 6493 games using it.


>6.a Who was the highest rated player for white?<br>
>6.b Who was the lowest rated player for white?<br>
>6.c Who was the highest rated player for black?<br>
>6.d Who was the lowest rated player for black?

In [16]:
white_ratings = []
black_ratings = []
for i in data:
    white_ratings.append(int(i["white_rating"]))
    black_ratings.append(int(i["black_rating"]))
white_ratings.sort()
black_ratings.sort()

highest_rated_white = ""
lowest_rated_white = ""
highest_rated_black = ""
lowest_rated_black = ""
for i in data:
    if int(i["white_rating"]) == white_ratings[len(white_ratings)-1]:
        highest_rated_white = i["white_id"]
    elif int(i["white_rating"]) == white_ratings[0]:
        lowest_rated_white = i["white_id"]
    if int(i["black_rating"]) == black_ratings[len(black_ratings)-1]:
        highest_rated_black = i["black_id"]
    elif int(i["black_rating"]) == black_ratings[0]:
        lowest_rated_black = i["black_id"]
    if highest_rated_white != "" and lowest_rated_white != "" and highest_rated_black != "" and highest_rated_black != "":
        break;
print("The highest rated player for white was " + highest_rated_white + " with a rating of " + str(white_ratings[len(white_ratings)-1]) + ".")
print("The lowest rated player for white was " + lowest_rated_white + " with a rating of " + str(white_ratings[0]) + ".")
print("The highest rated player for black was " + highest_rated_black + " with a rating of " + str(black_ratings[len(black_ratings)-1]) + ".")
print("The lowest rated player for black was " + lowest_rated_black + " with a rating of " + str(black_ratings[0]) + ".")

The highest rated player for white was blitzbullet with a rating of 2622.
The lowest rated player for white was ragnarlothbrook_spb with a rating of 784.
The highest rated player for black was avill050 with a rating of 2588.
The lowest rated player for black was hortense with a rating of 791.


>7.a What were the top 3 most played openings?<br>
>7.b How many moves was the longest opening played?<br>
>7.c How many moves was the shortest opening played?

In [17]:
openings = []
counters = []
for i in data:
    partition = i["opening_name"].split(":")
    new = True
    for j in range(len(openings)):
        if openings[j] == partition[0]:
            counters[j] += 1
            new = False
            break
    if new:
        openings.append(partition[0])
        counters.append(1)
most_moves = int(data[0]["opening_ply"])
most_name = data[0]["opening_name"]
least_moves = int(data[0]["opening_ply"])
least_name = data[0]["opening_name"]
for i in data:
    if int(i["opening_ply"]) > most_moves:
        most_moves = int(i["opening_ply"])
        most_name = i["opening_name"]
    elif int(i["opening_ply"]) < least_moves:
        least_moves = int(i["opening_ply"])
        least_name = i["opening_name"]
big_3 = []
big_3_counter = []
for i in range(3):
    biggest = counters[0];
    biggest_index = 0;
    for i in range(1, len(counters)):
        if counters[i] > biggest:
            biggest = counters[i]
            biggest_index = i
    big_3_counter.append(counters[biggest_index])
    big_3.append(openings[biggest_index])
    openings.pop(biggest_index)
    counters.pop(biggest_index)
top_3 = tuple(zip(big_3, big_3_counter))
message = "The top 3 openings played are: \n"
for i in range(3):
    message += "\t" + str(i+1) + ". The " + str(top_3[i][0]) + ", played " + str(top_3[i][1]) + " times.\n"
print(message)
print("The opening with the most moves is the " + most_name + " with " + str(most_moves) + " moves.")
print("The opening with the least moves is the " + least_name + " with " + str(least_moves) + " moves.")

The top 3 openings played are: 
	1. The Sicilian Defense, played 2012 times.
	2. The French Defense, played 1077 times.
	3. The Queen's Pawn Game, played 801 times.

The opening with the most moves is the Ruy Lopez: Marshall Attack |  Modern Main Line with 28 moves.
The opening with the least moves is the Queen's Pawn with 1 moves.


## <span style="color:blue"> Exploratory Data Analysis:</span>
>**1. Are the chances of winning the game greater if you are playing white? If so, why is that?**

>**2. Does the number of moves affect the outcome of the game? If the number of moves played goes over the average number of moves, what is the most likely outcome of the match? What if the number of moves is below average?**

>**3. How do the following victory statuses correlate with rating? Find the correlation coefficient and determine whether the relationship is strong/weak and positive/negative. What conclusions can be drawn from the data collected?**<br>
>  a) Checkmate<br>
>  b) Resignation<br>
>  c) Timeout <br>

>**4.a List the top 5 highest win rate openings, what are their win rates?<br>
4.b Do their win rates vary depending on piece colour?<br>
4.c List the top 5 players that use the combination of these openings the most, what are their win rates and rating?<br>
4.d What conclusions can be drawn from the data collected?**

>**5.a What is the relationship between the least and most common chess openings and their play rates?<br>
5.b Does the trend indicate increasing or decreasing volatility in victory results?<br>
5.c What conclusions can be drawn from the data collected?**

>**6. Is there a correlation between how many variations an opening has and how often it is played?**

## <span style="color:blue"> Potential Data Analysis:</span>

## <span style="color:blue"> Conclusion:</span>